## 🛠️ Mod5 Data Challenge 4: Funnels & Step Conversions 


### **Goal:** Build a simple funnel using Olist’s public marketing data:
1) **MQL**: all marketing qualified leads (MQLs)
2) **Contacted**: leads with a non-null `first_contact_date`
3) **Won**: leads that appear in `closed_deals` (i.e., `won_date` not null)

We’ll compute counts and conversion rates overall and by **origin** (channel).

### Data
A marketing dataset by the Olist Store (one-year time frame).  Two files (one with "leads" the other with "closed deals")  Read more about the data [HERE](https://www.kaggle.com/datasets/olistbr/marketing-funnel-olist/data?select=olist_marketing_qualified_leads_dataset.csv).   

### 👩‍🏫 Instructor-Led Demo (25 minutes)

#### Step 0:  Load both data files, format date columns, AND JOIN them together

In [63]:
import pandas as pd
import numpy as np

#mql is the marketing qualified leads and closed is the closed deals 

mql = pd.read_csv('../data/marketingleads.csv')
closed = pd.read_csv('../data/closeddeals.csv')
# Standardize date columns to datetime objects
mql['first_contact_date'] = pd.to_datetime(mql.get('first_contact_date'))
closed['won_date'] = pd.to_datetime(closed.get('won_date'))

# Join on mql_id to mark closed/won
funnel = pd.merge(mql, closed, on='mql_id', how="left")

# Robust channel column (named 'origin' in the dataset)
channel_col = 'origin' if 'origin' in funnel.columns else None
if channel_col is None:
    # if channel missing, create a single bucket
    funnel['origin'] = 'All'
    channel_col = 'origin'

funnel.head(3)

,mql_id,first_contact_date,landing_page_id,origin,seller_id,sdr_id,sr_id,won_date,business_segment,lead_type,lead_behaviour_profile,has_company,has_gtin,average_stock,business_type,declared_product_catalog_size,declared_monthly_revenue
0,dac32acd4db4c29c230538b72f8dd87d,2018-02-01,88740e65d5d6b056e0cda098e1ea6313,social,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8c18d1de7f67e60dbd64e3c07d7e9d5d,2017-10-20,007f9098284a86ee80ddeb25d53e0af8,paid_search,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,b4bc852d233dfefc5131f593b538befa,2018-03-22,a7982125ff7aa3b2054c6e44f9d28522,organic_search,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
funnel.columns

Index(['mql_id', 'first_contact_date', 'landing_page_id', 'origin',
       'seller_id', 'sdr_id', 'sr_id', 'won_date', 'business_segment',
       'lead_type', 'lead_behaviour_profile', 'has_company', 'has_gtin',
       'average_stock', 'business_type', 'declared_product_catalog_size',
       'declared_monthly_revenue'],
      dtype='object')

#### Step 1:  Define the Funnel


In [ ]:
# Binary flags for each step
funnel['is_mql'] = 1
funnel['is_contacted'] = funnel['first_contact_date'].notna().astype(int)
funnel['is_won'] = funnel['won_date'].notna().astype(int)

steps = ['is_mql', 'is_contacted', 'is_won']

In [65]:
funnel.shape

(8000, 20)

#### Step 2:  Create the Step-to-step conversion columns for an Overall Funnel (across ALL origins)

Students:  What does the `overall_conv` of `is_won` tell you?

In [68]:
overall = funnel[steps].sum().to_frame('count')
overall['step'] = overall.index.map({'is_mql':'MQL','is_contacted':'Contacted','is_won':'Won'})
overall = overall[['step','count']]

# Step-to-step conversion
overall['step_conv'] = overall['count'] / overall['count'].shift(1)
overall['overall_conv'] = overall['count'] / overall['count'].iloc[0]


# Overall conversion (from first step)
overall['step_conv'] = (overall['step_conv']*100).round(2)
overall['overall_conv'] = (overall['overall_conv'] * 100).round(2)
overall['overall_conv_perc'] = (overall['overall_conv']*100).round(2)


In [67]:
overall

,step,count,step_conv,overall_conv,overall_conv_perc
is_mql,MQL,8000,NaN,100.00,10000.0
is_contacted,Contacted,8000,100.00,100.00,10000.0
is_won,Won,842,10.52,10.52,1052.0


#### Step 3:  Create a Funnel by Channel (counts + rates)

Students:  What are some important insights from the results?  Would you recommend a marketing team to use one origin over the other? 

In [55]:
by_ch = (funnel
         .groupby(channel_col)[steps]
         .sum()
         .sort_values('is_mql', ascending=False))

# Compute rates per channel row
rates = by_ch.copy()
rates['contacted_rate'] = (rates['is_contacted'] / rates['is_mql']).round(3)
rates['won_rate']       = (rates['is_won'] / rates['is_mql']).round(3)
rates['overall_rate']   = (rates['is_won'] / rates['is_mql']).round(3)

# Tidy view
out = by_ch.join(rates[['contacted_rate','won_rate','overall_rate']])
out


,is_mql,is_contacted,is_won,contacted_rate,won_rate,overall_rate
origin,,,,,,
organic_search,2296,2296,271,1.0,0.118,0.118
paid_search,1586,1586,195,1.0,0.123,0.123
social,1350,1350,75,1.0,0.056,0.056
unknown,1099,1099,179,1.0,0.163,0.163
direct_traffic,499,499,56,1.0,0.112,0.112
email,493,493,15,1.0,0.030,0.030
referral,284,284,24,1.0,0.085,0.085
other,150,150,4,1.0,0.027,0.027
display,118,118,6,1.0,0.051,0.051


#### Instructor Section Notes: Interpreting the Output

- **Largest drop** is usually MQL → Contacted (sales team reach rate).
- **Won_rate** captures how many contacted leads ultimately close; it varies by origin.
- **Overall_rate** (Won/MQL) is the north-star for channel efficiency.
- Action: pick the **weakest step per channel** and state a concrete improvement (copy, targeting, SDR playbook, follow-up cadence, etc.).


### 👩‍💻 Student-Led Section (20 minutes) -- ANSWER KEY

### Student: Build & Interpret **Device** Funnels (Desktop vs Mobile)

**Your tasks:**
1) Make per-device funnel counts (MQL, Contacted, Won).
2) Compute step conversion rates and overall conversion per device.
3) Identify which device has the weakest "won_rate" and make a recommendation.  




In [84]:
# RUN THIS CELL WITHOUT CHANGES!  We are making a hypothetical "device" column to add to the data 

# Normalize/mend origin
if 'origin' not in funnel.columns:
    funnel['origin'] = 'Unknown'
funnel['origin'] = funnel['origin'].fillna('Unknown').astype(str)

# Channel → device priors (rough, realistic)
device_priors = {
    'Email':       {'Desktop': 0.60, 'Mobile': 0.40},
    'Paid Search': {'Desktop': 0.45, 'Mobile': 0.55},
    'Paid_Search': {'Desktop': 0.45, 'Mobile': 0.55},  # handle underscore variant
    'Social':      {'Desktop': 0.30, 'Mobile': 0.70},
    'Direct':      {'Desktop': 0.55, 'Mobile': 0.45},
    'Referral':    {'Desktop': 0.50, 'Mobile': 0.50},
    'Unknown':     {'Desktop': 0.50, 'Mobile': 0.50},
}

# Reproducible assignment
rng = np.random.default_rng(7)

def pick_device(origin):
    priors = device_priors.get(origin, device_priors['Unknown'])
    return rng.choice(['Desktop','Mobile'], p=[priors['Desktop'], priors['Mobile']])

funnel['device'] = funnel['origin'].apply(pick_device)

# Quick sanity check
funnel['device'].value_counts(normalize=True).round(3)

device
Mobile     0.501
Desktop    0.499
Name: proportion, dtype: float64

#### Task 1:  Get per-device counts


In [85]:
steps = ['is_mql','is_contacted','is_won']

# Make sure a device column exists; if not, default everything to "Unknown"
if 'device' not in funnel.columns:
    funnel['device'] = 'Unknown'

# Group and sum counts per device
funnel_counts_student_device = funnel.groupby('device')[steps].sum()  #Hint:  Group by device and steps
funnel_counts_student_device

,is_mql,is_contacted,is_won
device,,,
Desktop,3990,3990,425
Mobile,4010,4010,417


#### Task 2:  Get Per-Device Rates 


In [87]:
conv_student_device = funnel_counts_student_device.copy()

# Step rates
# contacted_rate = contacted / mql
# won_rate       = won / contacted
# overall_rate   = won / mql

conv_student_device['contacted_rate'] = conv_student_device['is_contacted'] / conv_student_device['is_mql']
conv_student_device['won_rate'] = conv_student_device['is_won'] / conv_student_device['is_contacted']  
conv_student_device['overall_rate'] = conv_student_device['is_won'] / conv_student_device['is_mql']    

# Show rates only (cleaner view)
conv_student_device[['contacted_rate','won_rate','overall_rate']]

,contacted_rate,won_rate,overall_rate
device,,,
Desktop,1.0,0.106516,0.106516
Mobile,1.0,0.103990,0.103990


In [88]:
conv_student_device

,is_mql,is_contacted,is_won,contacted_rate,won_rate,overall_rate
device,,,,,,
Desktop,3990,3990,425,1.0,0.106516,0.106516
Mobile,4010,4010,417,1.0,0.103990,0.103990


#### Task 3: Weakest Step Per Device

In [ ]:
# For each device, find the smallest of contacted_rate and won_rate
rate_cols = ['contacted_rate','won_rate']

weakest_step_device = conv_student_device[rate_cols].idxmin(axis=1) #Hint use a .idxmin() on the rate_cols
weakest_rate_device = conv_student_device[rate_cols].min(axis=1)

# Combine into a tidy summary
summary_device = conv_student_device[['contacted_rate','won_rate','overall_rate']].copy()
summary_device['weakest_step'] = weakest_step_device
summary_device['weakest_rate'] = weakest_rate_device.round(3)
summary_device

,contacted_rate,won_rate,overall_rate,weakest_step,weakest_rate
device,,,,,
Desktop,1.0,0.106516,0.106516,won_rate,0.107
Mobile,1.0,0.103990,0.103990,won_rate,0.104


In [89]:
weakest_step_device

device
Desktop    won_rate
Mobile     won_rate
dtype: object

### Interpretation of Your Results (3-5 Sentences)

- For **Desktop** and **Mobile**, name the device that had the weakest "won_rate"
- Suggest **one device‑specific action**:
  - For Example:  "If Desktop contacted_rate is low → test desktop hero/CTA (call to action) placement"
- Name **one metric** you’ll monitor next week to confirm improvements (e.g., won_rate for Mobile).


  - The two type of devices have almost the same won_rate, but the weakest one is `Mobile` with 10.39% compare to `Desktope` with 10.65% the Desktop device a slighty higher won rate. One specific action to take for device would be making sure that the process to step that users take is reduce because, people who use phone are multi-tasking if the process is long the will drop-off. One metric I could track user's won_rate on mobile and compare it to Desktop again by doing a funnel analysis to see if there is change, by doing so i'll see exactly where the drop-off happens. 


### 📣 Class Share-Out & Instructor Wrap-Up (15 minutes)

Be ready share out the following points with the class: 

**Explain:**
Your one device-specific action and the one metric you would monitor 

#### Instructor Wrap-Up (Notes)


- **Different segmentation, different story**: Channel vs Device can surface different weak steps in marketing. Keep both lenses in your toolkit.
- **Targeted fixes**:
  - Low *contacted_rate* → acquisition UX: ad/landing alignment, page speed (esp. mobile), CTA placement.
  - Low *won_rate* → checkout/closing UX: form length, trust badges, mobile wallets, pricing clarity.
- **Explainability first**: Step rates are intuitive and align to concrete product or marketing experiments.
- **Guardrails**:
  - Ensure steps belong to the same journey window (avoid mixing old impressions with recent sales).
  - Dedupe MQLs; validate contact timestamps (no “phantom” contacts).
  - Consider seasonality and campaigns when comparing devices.
- **Next**:
  - Cross‑segmentation (Device × Channel) to find where the *biggest leaks* are.
  - Time trends: weekly device funnels to catch regressions after UI changes.
  - Tie experiments to a single target step (e.g., Mobile *won_rate*) and commit to a monitoring window.




In [90]:
funnel

,mql_id,first_contact_date,landing_page_id,origin,seller_id,sdr_id,sr_id,won_date,business_segment,lead_type,...,has_company,has_gtin,average_stock,business_type,declared_product_catalog_size,declared_monthly_revenue,is_mql,is_contacted,is_won,device
0,dac32acd4db4c29c230538b72f8dd87d,2018-02-01,88740e65d5d6b056e0cda098e1ea6313,social,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,Mobile
1,8c18d1de7f67e60dbd64e3c07d7e9d5d,2017-10-20,007f9098284a86ee80ddeb25d53e0af8,paid_search,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,Mobile
2,b4bc852d233dfefc5131f593b538befa,2018-03-22,a7982125ff7aa3b2054c6e44f9d28522,organic_search,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,Mobile
3,6be030b81c75970747525b843c1ef4f8,2018-01-22,d45d558f0daeecf3cccdffe3c59684aa,email,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,Desktop
4,5420aad7fec3549a85876ba1c529bd84,2018-02-21,b48ec5f3b04e9068441002a19df93c6c,organic_search,2c43fb513632d29b3b58df74816f1b06,a8387c01a09e99ce014107505b92388c,4ef15afb4b2723d8f3d81e51ec7afefe,2018-02-26 19:58:54,pet,online_medium,...,NaN,NaN,NaN,reseller,NaN,0.0,1,1,1,Desktop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,feaba3ffcd2ff97501696c7f9a42f41c,2018-05-22,e42a14209c69c3e9cc6b042620465f12,paid_search,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,Desktop
7996,a79cb53cd009ab92e0143b92baa2407b,2018-03-27,c494978688ccf66ad9fad3d6a3338c22,paid_search,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,Mobile
7997,68f049a23ab109c6a0f6989bb9a02994,2017-08-27,b48ec5f3b04e9068441002a19df93c6c,organic_search,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,Desktop
7998,4f8c96e2509b984329044c6682c88ee9,2017-10-06,a56671a54260a44923d32c2f08fad39c,organic_search,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,Mobile
